# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column

y = loans_df["credit_ranking"].values

# Display a sample of y

y[:5]

array([0, 0, 0, 1, 0], dtype=int64)

In [6]:
# Define features set X by selecting all columns but credit_ranking

X = loans_df.drop(columns=["credit_ranking"])

# Review the features DataFrame

X.head()


,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance

scaler = StandardScaler()

# Fit the scaler to the features training dataset

scaler.fit(X_train)

# Fit the scaler to the features training dataset

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [9]:
# Define the the number of inputs (features) to the model

number_input_features = X_train_scaled.shape[1]

# Review the number of features

number_input_features

11

In [10]:
# Define the number of hidden nodes for the first hidden layer

number_hidden_nodes_layer1 = 6


# Define the number of hidden nodes for the second hidden layer

number_hidden_nodes_layer2 = 3

# Define the number of neurons in the output layer

number_output_neurons = 1

In [11]:
# Create the Sequential model instance

nn_model = Sequential()

# Add the first hidden layer

nn_model.add(Dense(units=number_hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Add the second hidden layer

nn_model.add(Dense(units=number_hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function

nn_model.add(Dense(units=number_output_neurons, activation="sigmoid"))

In [12]:
# Display the Sequential model summary

nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 3)                 21        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97
Trainable params: 97
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [13]:
# Compile the Sequential model

nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Fit the model using 50 epochs and the training data

fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, shuffle=True, verbose=2)

Epoch 1/50
38/38 - 1s - loss: 0.6831 - accuracy: 0.5763 - 513ms/epoch - 13ms/step
Epoch 2/50
38/38 - 0s - loss: 0.6633 - accuracy: 0.6063 - 28ms/epoch - 725us/step
Epoch 3/50
38/38 - 0s - loss: 0.6490 - accuracy: 0.6230 - 26ms/epoch - 685us/step
Epoch 4/50
38/38 - 0s - loss: 0.6352 - accuracy: 0.6439 - 27ms/epoch - 699us/step
Epoch 5/50
38/38 - 0s - loss: 0.6219 - accuracy: 0.6580 - 26ms/epoch - 685us/step
Epoch 6/50
38/38 - 0s - loss: 0.6092 - accuracy: 0.6722 - 27ms/epoch - 698us/step
Epoch 7/50
38/38 - 0s - loss: 0.5978 - accuracy: 0.6872 - 27ms/epoch - 698us/step
Epoch 8/50
38/38 - 0s - loss: 0.5873 - accuracy: 0.6981 - 27ms/epoch - 712us/step
Epoch 9/50
38/38 - 0s - loss: 0.5782 - accuracy: 0.6981 - 27ms/epoch - 699us/step
Epoch 10/50
38/38 - 0s - loss: 0.5703 - accuracy: 0.7073 - 28ms/epoch - 725us/step
Epoch 11/50
38/38 - 0s - loss: 0.5633 - accuracy: 0.7148 - 28ms/epoch - 738us/step
Epoch 12/50
38/38 - 0s - loss: 0.5573 - accuracy: 0.7131 - 27ms/epoch - 698us/step
Epoch 13/50
3

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [15]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data

model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 0.5102 - accuracy: 0.7600 - 152ms/epoch - 12ms/step
Loss: 0.5102166533470154, Accuracy: 0.7599999904632568


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [16]:
# Set the model's file path

model_file_path = Path("student_loans_model.keras")

# Export your model to a keras file

nn_model.save(model_file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [17]:
# Set the model's file path

model_file_path = Path("student_loans_model.keras")

# Load the model to a new object

loaded_model = tf.keras.models.load_model(model_file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [18]:
# Make predictions with the test data

predictions = nn_model.predict(X_test_scaled)

# Display a sample of the predictions

predictions[:5]

13/13 [==============================] - 0s 709us/step


array([[0.36948988],
       [0.3247764 ],
       [0.8636079 ],
       [0.77932954],
       [0.9860765 ]], dtype=float32)

In [19]:
# Save the predictions to a DataFrame and round the predictions to binary results

predictions_df = pd.DataFrame(predictions, columns=["Predictions"])
predictions_df["Predictions"] = predictions_df["Predictions"].round(0)
predictions_df

,Predictions
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [20]:
# Print the classification report with the y test data and predictions

print(classification_report(y_test, predictions_df["Predictions"]))

              precision    recall  f1-score   support

           0       0.74      0.76      0.75       188
           1       0.78      0.76      0.77       212

    accuracy                           0.76       400
   macro avg       0.76      0.76      0.76       400
weighted avg       0.76      0.76      0.76       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

To build a recommendation system for student loan options, the data needed would include:

    Student Demographics: Age, location, and income level to understand the financial background.
    Academic Information: GPA, study major, and time to completion to assess academic performance and potential.
    Loan History: Payment history, total loan score, and financial aid score to evaluate financial behavior.
    Extracurricular Activities: Participation in finance workshops or alumni success metrics to gauge financial literacy and career prospects.

    This data is relevant as it provides a comprehensive view of the student's financial and academic profile, enabling personalized loan recommendations.

    The data described aligns well with the features available in the dataset `X`, which includes attributes such as `payment_history`, `gpa_ranking`, `financial_aid_score`, and others. These features can be used to build a recommendation system by leveraging machine learning models to predict suitable loan options based on individual student profiles.
    

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

    Based on the data, the model would use **content-based filtering**. This is because the recommendations are based on the student's individual attributes (e.g., GPA, financial aid score) rather than relying on the preferences or behaviors of other users (collaborative filtering). I rejected Context-based filtering since typically incorporates external contextual information, such as time, location, or specific circumstances, to make recommendations. While the dataset includes features like location_parameter and time_to_completion, these are intrinsic attributes of the students rather than external contextual factors. Therefore, the data aligns more closely with content-based filtering, which focuses on the attributes of the items or users themselves.

    The choice of content-based filtering was appropriate because the data selected focuses on individual attributes of the students, such as GPA ranking, financial aid score, and payment history. These features are intrinsic to each student and allow the recommendation system to provide personalized loan options based on their unique profiles. Unlike collaborative filtering, which relies on user interactions or preferences, content-based filtering leverages the specific characteristics of the data to make recommendations. This aligns well with the dataset's structure and the goal of tailoring loan recommendations to individual needs.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

    Two real-world challenges to consider:
        Data Privacy and Security: Collecting sensitive financial and academic data requires robust measures to protect student privacy and comply with regulations like GDPR.
        Bias in Data: Historical biases in loan approvals or academic performance metrics could lead to unfair recommendations, necessitating careful preprocessing and validation of the data.